In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from nuztf.neutrino_scanner import NeutrinoScanner
from astropy import units as u
from nuwinter.avro import load_avro
from nuwinter.plot import ann_fields, generate_single_page
from nuwinter.utils import deduplicate_df
from nuwinter.paths import get_pdf_path
from pathlib import Path
import pandas as pd
import json
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from tqdm import tqdm
from nuztf.skymap_scanner import SkymapScanner







/Users/robertstein/anaconda3/envs/nuwinter_env/lib/python3.11/site-packages/ligo/lw/lsctables.py:89: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
base_dir = Path.home().joinpath("Data/winter")

# name = "IC240412A"

name = "S240422ed"

prob_threshold = 0.95
n_days = 14

In [3]:
outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
outpath

'/Users/robertstein/Data/nuwinter/S240422ed/winter_candidate_pdf/combined_ztf.pdf'

In [4]:
nu = NeutrinoScanner(name) if "IC" in name else SkymapScanner(
    event=name,
    prob_threshold=prob_threshold,
    n_days=n_days
)

100%|█████████████████████████████████████| 435/435 [00:00<00:00, 245495.46it/s]


In [5]:
ztf_initial_cache = nu.get_initial_cache_path()

download_ztf = False

if not (ztf_initial_cache.exists()):
    download_ztf = True

if ztf_initial_cache.exists():
    if not ztf_initial_cache.stat().st_size > 2:
        download_ztf = True

if download_ztf:
    print(f"No file found at {ztf_initial_cache}")
    if isinstance(nu, SkymapScanner):
        nu.get_alerts()
        nu.filter_alerts()
    else:
        nu.scan_area(t_max=nu.t_min + 14.)

ztf_initial_cache

PosixPath('/Users/robertstein/Data/nuztf/cache/candidates/S240422ed/rev4/0.95/initial_stage.json')

In [6]:
nights = [str(nu.t_min + (i - 1) * u.day).split("T")[0].replace("-", "") for i in range(n_days + 1)]

all_res = []

for night in sorted(nights):
    
    avro_dir = base_dir / f"{night}/avro"

    print(f"Looking for avros in {avro_dir}")
    
    avro_files = list(avro_dir.glob("*.avro"))
    
    for path in avro_files:
        all_res.append(load_avro(path))

if len(all_res) == 0:
    raise FileNotFoundError(f"No avro files found for any night, please download these first using the download notebook!")

res_winter = pd.concat(all_res)

Looking for avros in /Users/robertstein/Data/winter/20240421/avro
Looking for avros in /Users/robertstein/Data/winter/20240422/avro
Looking for avros in /Users/robertstein/Data/winter/20240423/avro
Looking for avros in /Users/robertstein/Data/winter/20240424/avro
Looking for avros in /Users/robertstein/Data/winter/20240425/avro
Looking for avros in /Users/robertstein/Data/winter/20240426/avro
Looking for avros in /Users/robertstein/Data/winter/20240427/avro
Looking for avros in /Users/robertstein/Data/winter/20240428/avro
Looking for avros in /Users/robertstein/Data/winter/20240429/avro
Looking for avros in /Users/robertstein/Data/winter/20240430/avro
Looking for avros in /Users/robertstein/Data/winter/20240501/avro
Looking for avros in /Users/robertstein/Data/winter/20240502/avro
Looking for avros in /Users/robertstein/Data/winter/20240503/avro
Looking for avros in /Users/robertstein/Data/winter/20240504/avro
Looking for avros in /Users/robertstein/Data/winter/20240505/avro


/var/folders/hd/1411jr114w9cvmmnw9mx0xvr0000gn/T/ipykernel_30263/3059914597.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res_winter = pd.concat(all_res)


In [7]:
in_contour = []

for _, row in res_winter.iterrows():
    in_contour.append(nu.in_contour(row["ra"], row["dec"]))

print(f"{np.sum(in_contour)} / {len(in_contour)} candidates in contour")

res_winter = res_winter[in_contour]

66351 / 74896 candidates in contour


In [8]:
res_winter

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmjmag3,tmhmag3,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright
0,WNTR24bxplv,b'\x1f\x8b\x08\x00q\x83*f\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00q\x83*f\x02\xff\xed\xbcgPT]\...,b'\x1f\x8b\x08\x00q\x83*f\x02\xff\xec\xdci4\xd...,candid progname jd fid isdif...,4152416,False,2.460425e+06,2,480.0,...,14.785000,14.587000,14.385,08140955-2555343,2.832647,18.625593,0.973633,37.919456,18.431753,1.029493
1,WNTR24bxplz,b'\x1f\x8b\x08\x00t\x83*f\x02\xff\xec\x97iT\xc...,b'\x1f\x8b\x08\x00t\x83*f\x02\xff\xed\xbbyP\x8...,b'\x1f\x8b\x08\x00t\x83*f\x02\xff\xec\xbdWP\x9...,candid progname jd fid isdif...,4152419,False,2.460425e+06,2,480.0,...,16.537001,16.315001,16.684,08140662-2556504,1.595792,7.822520,1.008900,45.290405,24.733997,1.207167
2,WNTR24bxpmd,b'\x1f\x8b\x08\x00u\x83*f\x02\xff\xec\x97i4\x9...,b'\x1f\x8b\x08\x00u\x83*f\x02\xff\xed\xbbWP\x1...,b'\x1f\x8b\x08\x00u\x83*f\x02\xff\xec\xdcUT\x9...,candid progname jd fid isdif...,4152422,False,2.460425e+06,2,480.0,...,11.109000,10.570000,10.302,08140831-2559002,1.587264,16.576431,1.019907,20.705017,80.259789,11.776192
3,WNTR24bxpmg,b'\x1f\x8b\x08\x00v\x83*f\x02\xff\xec\x97\xf9W...,b'\x1f\x8b\x08\x00v\x83*f\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00v\x83*f\x02\xff\xec\x97i4\xd...,candid progname jd fid isdif...,4152423,False,2.460425e+06,2,480.0,...,14.957000,14.621000,14.423,08142578-2600240,4.930763,2.042831,1.060340,NaN,NaN,NaN
4,WNTR24bxpmr,b'\x1f\x8b\x08\x00\x85\x83*f\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00\x85\x83*f\x02\xff\xec\x97i4...,b'\x1f\x8b\x08\x00\x85\x83*f\x02\xff\xecwi4\xd...,candid progname jd fid isdif...,4152435,False,2.460425e+06,2,480.0,...,14.344000,14.101000,13.869,08143107-2601130,8.868516,1.328499,1.018889,50.776855,8.935471,16.825293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,WNTR24cihqk,b'\x1f\x8b\x08\x00\xb4w2f\x02\xff\xed\xd7y0\x9...,b'\x1f\x8b\x08\x00\xb4w2f\x02\xff\xed\xb7yT\x8...,b'\x1f\x8b\x08\x00\xb4w2f\x02\xff\xec\xd7i4\xd...,candid deprecated jd fid exp...,4310795,False,2.460432e+06,2,360.0,...,16.534000,15.868000,14.579,08180865-2108405,6.222070,27.828482,0.972898,24.537914,89.026703,0.973026
682,WNTR24cihql,b'\x1f\x8b\x08\x00\xb5w2f\x02\xff\xed\xday0\x9...,b'\x1f\x8b\x08\x00\xb5w2f\x02\xff\xed\xd7i4\x9...,b'\x1f\x8b\x08\x00\xb5w2f\x02\xff\xec\xd7UP\x9...,candid deprecated jd fid exp...,4310796,False,2.460432e+06,2,360.0,...,15.042000,14.592000,14.623,08180096-2108535,10.132645,1.192362,0.939409,75.350250,70.399963,1.024324
683,WNTR24cihqm,b'\x1f\x8b\x08\x00\xb5w2f\x02\xff\xed\xd7yP\x8...,b'\x1f\x8b\x08\x00\xb5w2f\x02\xff\xed\x97iT\x8...,b'\x1f\x8b\x08\x00\xb5w2f\x02\xff\xec\xd7y4\xd...,candid deprecated jd fid exp...,4310797,False,2.460432e+06,2,360.0,...,NaN,NaN,NaN,nan,2.947953,1.922692,2.372059,30.222391,89.026703,0.973026
684,WNTR24cfswj,b'\x1f\x8b\x08\x00\xcfy2f\x02\xff\xec\x9byP\xc...,b'\x1f\x8b\x08\x00\xcfy2f\x02\xff\xec\xbcy4\x9...,b'\x1f\x8b\x08\x00\xcfy2f\x02\xff\xec\x9aWL\x1...,candid progname jd fid isdif...,4310798,False,2.460432e+06,2,360.0,...,NaN,NaN,NaN,nan,12.280168,4.592246,1.064449,81.835052,31.495352,1.303939


In [9]:
with open(ztf_initial_cache, "r") as f:
    res_ztf = json.load(f)
    
ztf_sources = []

for entry in res_ztf:

    new_dict = {}
    
    for field in ["candid", "objectId"]:
        new_dict[field] = entry[field]

    for key, val in entry["candidate"].items():
        new_dict[key] = val

    ztf_sources.append(new_dict)

ztf_df = pd.DataFrame(ztf_sources)
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,programid,isdiffpos,tblid,nid,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,2669140023715070003,ZTF24aakxgha,2.460424e+06,2,2669140023715,20.152534,2,f,3,2669,...,28.742001,0.502,0.179616,0.148171,77.994057,15.800767,12.335743,300.0,0.273111,d6_m7
1,2669140025215070048,ZTF24aakxgku,2.460424e+06,2,2669140025215,19.955992,2,f,48,2669,...,28.688999,0.462,0.139645,0.314258,68.333260,15.153409,13.428930,300.0,0.350037,d6_m7
2,2669143614115015033,ZTF24aakvrhi,2.460424e+06,2,2669143614115,20.485649,2,t,33,2669,...,28.868000,0.541,0.152793,0.063140,33.398094,17.630253,8.452576,300.0,0.178979,d6_m7
3,2669140024415070050,ZTF24aakxgac,2.460424e+06,2,2669140024415,20.397671,2,f,50,2669,...,28.806000,0.585,0.160592,0.453580,-999.000000,16.274351,-999.000000,300.0,0.064933,d6_m7
4,2669140025115070012,ZTF24aakxgma,2.460424e+06,2,2669140025115,19.988226,2,f,12,2669,...,28.648001,0.433,0.135750,0.100210,52.195656,17.820419,10.715789,300.0,0.998985,d6_m7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25647,2677235984215015079,ZTF24aadpcuf,2.460432e+06,1,2677235984215,20.516619,2,t,79,2677,...,28.266500,0.463,0.193121,0.955089,45.867584,14.544926,12.818110,300.0,0.067297,d6_m7
25648,2677235984515015174,ZTF23abtkixz,2.460432e+06,1,2677235984515,20.415510,2,t,174,2677,...,28.191999,0.461,0.168263,0.851971,42.200748,14.086395,13.415142,300.0,0.203888,d6_m7
25649,2677235982115010013,ZTF24aalnqqn,2.460432e+06,1,2677235982115,20.539330,2,f,13,2677,...,28.230000,0.465,0.186417,1.261331,47.226009,16.706146,13.641432,300.0,0.914188,d6_m7
25650,2677239570215010065,ZTF24aalntlf,2.460432e+06,1,2677239570215,20.513945,2,f,65,2677,...,28.214001,0.494,0.208718,1.019851,80.081467,14.361336,12.831529,300.0,0.902978,d6_m7


In [10]:
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,programid,isdiffpos,tblid,nid,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,2669140023715070003,ZTF24aakxgha,2.460424e+06,2,2669140023715,20.152534,2,f,3,2669,...,28.742001,0.502,0.179616,0.148171,77.994057,15.800767,12.335743,300.0,0.273111,d6_m7
1,2669140025215070048,ZTF24aakxgku,2.460424e+06,2,2669140025215,19.955992,2,f,48,2669,...,28.688999,0.462,0.139645,0.314258,68.333260,15.153409,13.428930,300.0,0.350037,d6_m7
2,2669143614115015033,ZTF24aakvrhi,2.460424e+06,2,2669143614115,20.485649,2,t,33,2669,...,28.868000,0.541,0.152793,0.063140,33.398094,17.630253,8.452576,300.0,0.178979,d6_m7
3,2669140024415070050,ZTF24aakxgac,2.460424e+06,2,2669140024415,20.397671,2,f,50,2669,...,28.806000,0.585,0.160592,0.453580,-999.000000,16.274351,-999.000000,300.0,0.064933,d6_m7
4,2669140025115070012,ZTF24aakxgma,2.460424e+06,2,2669140025115,19.988226,2,f,12,2669,...,28.648001,0.433,0.135750,0.100210,52.195656,17.820419,10.715789,300.0,0.998985,d6_m7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25647,2677235984215015079,ZTF24aadpcuf,2.460432e+06,1,2677235984215,20.516619,2,t,79,2677,...,28.266500,0.463,0.193121,0.955089,45.867584,14.544926,12.818110,300.0,0.067297,d6_m7
25648,2677235984515015174,ZTF23abtkixz,2.460432e+06,1,2677235984515,20.415510,2,t,174,2677,...,28.191999,0.461,0.168263,0.851971,42.200748,14.086395,13.415142,300.0,0.203888,d6_m7
25649,2677235982115010013,ZTF24aalnqqn,2.460432e+06,1,2677235982115,20.539330,2,f,13,2677,...,28.230000,0.465,0.186417,1.261331,47.226009,16.706146,13.641432,300.0,0.914188,d6_m7
25650,2677239570215010065,ZTF24aalntlf,2.460432e+06,1,2677239570215,20.513945,2,f,65,2677,...,28.214001,0.494,0.208718,1.019851,80.081467,14.361336,12.831529,300.0,0.902978,d6_m7


In [11]:
if len(ztf_df) > 0:
    ztf_positions = SkyCoord(ztf_df["ra"].to_numpy()*u.deg, ztf_df["dec"].to_numpy()*u.deg, frame='icrs')
    
    winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')
    idx, d2d, _ = winter_positions.match_to_catalog_sky(ztf_positions)
    
    crossmatch_radius = 3.0 * u.arcsec
    mask = d2d < crossmatch_radius
    
    cut = res_winter[mask]
    cut.reset_index(inplace=True)
    cut

In [12]:
if len(ztf_df) > 0:
    ztf_names = []

    for i in idx[mask]:
        ztf_names.append(ztf_df["objectId"].iloc[i])
    
    cut = cut.assign(ztf_name=ztf_names)
    if len(cut) > 0:
        cut = deduplicate_df(cut)
    cut
else:
    cut = pd.DataFrame()

In [13]:
if len(cut) > 0:
    outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
    
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut.iterrows(), total=len(cut)):
            generate_single_page(row, ann_fields=["ztf_name"] + ann_fields)
            pdf.savefig()
            plt.close()

    outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf_crossmatched")

    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut.iterrows(), total=len(cut)):
            generate_single_page(row, ann_fields=["ztf_name"] + ann_fields, crossmatch=True)
            pdf.savefig()
            plt.close()
else:
    print("No candidates found")

Saving PDF to /Users/robertstein/Data/nuwinter/S240422ed/winter_candidate_pdf/combined_ztf.pdf


100%|███████████████████████████████████████████| 48/48 [00:02<00:00, 18.21it/s]


In [14]:
len(res_winter["objectid"])

66351

In [15]:
# crossmatch_radius = 2.0 * u.arcsec

# winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')


# match = []

# for i, c in enumerate(winter_positions):
#     if i < len(winter_positions) - 1:
#         idx, d2d, _ = c.match_to_catalog_sky(winter_positions[i+1:])
#         if d2d < crossmatch_radius:
#             match.append(idx)
#         else:
#             match.append(None)
#     else:
#         match.append(None)

In [16]:
# res_winter["match"] = match
# res_winter_match = res_winter[pd.notnull(match)]
res_winter_match = res_winter

In [17]:
# all_matched = []

# for i, row in res_winter_match.iterrows():
#     c = SkyCoord(row["ra"]*u.deg, row["dec"]*u.deg, frame='icrs')
#     sep = c.separation(winter_positions)
#     mask = (sep.arcsec > 0.) & (sep.arcsec < crossmatch_radius.value)
#     matches = res_winter.loc[mask]
#     row["ndethist"] = mask.sum()
#     row["prv_candidates"] = res_winter.loc[mask]
#     print(row["objectid"], res_winter.loc[mask]["objectid"])
#     all_matched.append(row)

# if len(all_matched) > 0:
#     res_winter_match = pd.concat(all_matched, axis=1).transpose()
#     res_winter_match

In [18]:
sgscore_cut = 0.5
dist_cut_arcsec = 5.

star_cut = (pd.notnull(res_winter_match["distpsnr1"]) & (res_winter_match["distpsnr1"] < dist_cut_arcsec) & pd.notnull(res_winter_match["sgscore1"]) & (res_winter_match["sgscore1"] > sgscore_cut))

masks = [
    res_winter_match["ndethist"] > 0,
    ~star_cut,
    res_winter_match["nbad"] < 2,
    res_winter_match["chipsf"] < 3.0,
    res_winter_match["sumrat"] > 0.7,
    res_winter_match["fwhm"] < 10.0,
    res_winter_match["magdiff"] < 1.6,
    res_winter_match["magdiff"] > -1.0,
    res_winter_match["mindtoedge"] > 50.0,
]

mask = np.ones(len(res_winter_match), dtype=bool)
for new_mask in masks:
    mask *= new_mask
    
print(f"{np.sum(mask)} / {len(mask)} detections pass all cuts")
cut2 = res_winter_match[mask]

if len(cut2) > 0:
    cut2 = deduplicate_df(cut2)
    cut2.sort_values(by="objectid", inplace=True)
    
    outpath = str(get_pdf_path(name)).replace("combined", "combined_2dets")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut2.iterrows(), total=len(cut2)):
            generate_single_page(row, ann_fields=ann_fields, crossmatch=False)
            pdf.savefig()
            plt.close()

    outpath = str(get_pdf_path(name)).replace("combined", "combined_2dets_crossmatched")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut2.iterrows(), total=len(cut2)):
            generate_single_page(row, ann_fields=ann_fields, crossmatch=True)
            pdf.savefig()
            plt.close()

203 / 66351 detections pass all cuts
Saving PDF to /Users/robertstein/Data/nuwinter/S240422ed/winter_candidate_pdf/combined_2dets.pdf


100%|█████████████████████████████████████████| 160/160 [00:09<00:00, 16.32it/s]


Saving PDF to /Users/robertstein/Data/nuwinter/S240422ed/winter_candidate_pdf/combined_2dets_crossmatched.pdf


100%|█████████████████████████████████████████| 160/160 [00:09<00:00, 16.49it/s]


In [19]:
list(res_winter_match.columns)

['objectid',
 'cutout_science',
 'cutout_template',
 'cutout_difference',
 'prv_candidates',
 'candid',
 'deprecated',
 'jd',
 'fid',
 'exptime',
 'ndethist',
 'jdstarthist',
 'jdendhist',
 'progname',
 'programid',
 'isdiffpos',
 'fieldid',
 'ra',
 'dec',
 'magzpsci',
 'magzpsciunc',
 'magzpscirms',
 'diffmaglim',
 'magpsf',
 'sigmapsf',
 'chipsf',
 'magap',
 'sigmagap',
 'magapbig',
 'sigmagapbig',
 'magdiff',
 'magfromlim',
 'distnr',
 'magnr',
 'sigmagnr',
 'xpos',
 'ypos',
 'sky',
 'fwhm',
 'classtar',
 'mindtoedge',
 'seeratio',
 'aimage',
 'bimage',
 'aimagerat',
 'bimagerat',
 'elong',
 'nneg',
 'nbad',
 'sumrat',
 'dsnrms',
 'ssnrms',
 'dsdiff',
 'scorr',
 'clrcoeff',
 'clrcounc',
 'zpclrcov',
 'zpmed',
 'clrmed',
 'clrrms',
 'rb',
 'rbversion',
 'ssdistnr',
 'ssmagnr',
 'ssnamenr',
 'tooflag',
 'nmtchps',
 'psra1',
 'psdec1',
 'psobjectid1',
 'sgmag1',
 'srmag1',
 'simag1',
 'szmag1',
 'sgscore1',
 'distpsnr1',
 'psobjectid2',
 'sgmag2',
 'srmag2',
 'simag2',
 'szmag2',
 'sgs